In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [2]:
df1 = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
#df1["proprosion"] = np.where(df1["cant_viewed_product"]>0, (df1["cant_checkouts"]/df1["cant_viewed_product"]),0)


In [3]:
df2 = pd.read_csv('data/features_time.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('data/sku.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('data/features_compras.csv', low_memory=False, index_col='person')


######################################################################################

df5 = pd.read_csv('data/vistas_promedio.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('data/featureUsuarioRealizaAlMenos10EventosEn20Minutos.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('data/features_mas_vistos.csv', low_memory=False, index_col='person')

df8 = pd.read_csv('data/features_checkout.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('data/paises_features.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('data/city_features.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('data/static.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('data/events_fe.csv', low_memory=False, index_col='person')
df13 = pd.read_csv('data/channel_device.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('data/check.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('data/view_sku.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('data/event_por_hora.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('data/ult_hora.csv', low_memory=False, index_col='person')


df_unidos = df1.join(df2).fillna(0)
df_unidos = df_unidos.join(df3).fillna(0)
df_unidos = df_unidos.join(df4).fillna(0)
df_unidos = df_unidos.join(df5).fillna(0)
df_unidos = df_unidos.join(df6).fillna(0)
df_unidos = df_unidos.join(df7).fillna(0)
df_unidos = df_unidos.join(df8).fillna(0)
#df_unidos = df_unidos.join(df9).fillna(0)
df_unidos = df_unidos.join(df10).fillna(0)
df_unidos = df_unidos.join(df11).fillna(0)
df_unidos = df_unidos.join(df12).fillna(0)
df_unidos = df_unidos.join(df13).fillna(0)
df_unidos = df_unidos.join(df14).fillna(0)
df_unidos = df_unidos.join(df15).fillna(0)
df_unidos = df_unidos.join(df16).fillna(0)
df_unidos = df_unidos.join(df17).fillna(0)

df_unidos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38829 entries, 4886f805 to 80aea0a0
Data columns (total 99 columns):
cant_conversions                                38829 non-null float64
cant_checkouts                                  38829 non-null float64
cant_viewed_product                             38829 non-null float64
cant_searched_product                           38829 non-null float64
cant_ad_campaign_hit                            38829 non-null float64
ad_campaign_hit                                 38829 non-null bool
cant_lead                                       38829 non-null float64
lead                                            38829 non-null bool
cant_brand_listing                              38829 non-null float64
cant_visited_site                               38829 non-null float64
cant_generic_listing                            38829 non-null float64
total_sesiones                                  38829 non-null int64
promedio_eventos_por_sesion               

In [4]:
labels = pd.read_csv('data/labels_training_set.csv', low_memory=False, index_col='person')

df_test = df_unidos.join(labels, how= "inner")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, ad93850f to d8d6b0da
Data columns (total 100 columns):
cant_conversions                                19414 non-null float64
cant_checkouts                                  19414 non-null float64
cant_viewed_product                             19414 non-null float64
cant_searched_product                           19414 non-null float64
cant_ad_campaign_hit                            19414 non-null float64
ad_campaign_hit                                 19414 non-null bool
cant_lead                                       19414 non-null float64
lead                                            19414 non-null bool
cant_brand_listing                              19414 non-null float64
cant_visited_site                               19414 non-null float64
cant_generic_listing                            19414 non-null float64
total_sesiones                                  19414 non-null int64
promedio_eventos_por_sesion              

In [5]:
features = df_test.columns.tolist()
features.remove('label')
print(features)

['cant_conversions', 'cant_checkouts', 'cant_viewed_product', 'cant_searched_product', 'cant_ad_campaign_hit', 'ad_campaign_hit', 'cant_lead', 'lead', 'cant_brand_listing', 'cant_visited_site', 'cant_generic_listing', 'total_sesiones', 'promedio_eventos_por_sesion', 'timestamp', 'min_timestamp', 'sum_timestamp', 'prom_timestamp', 'ult_view', 'ult_check', 'mean_sku', 'min_sku', 'suma_sku', 'compro_apple', 'compro_samsung', 'compro_motorola', 'cant_view_prodc', 'mas_prom', 'conte_10', 'vio_mas_apple', 'vio_mas_samsung', 'vio_mas_motorola', 'check_apple', 'check_samsung', 'check_motorola', 'city_Sao Paulo', 'city_S\xc3\xa3o Paulo', 'staticpage_AboutUs', 'staticpage_Conditions', 'staticpage_CustomerService', 'staticpage_FaqEcommerce', 'staticpage_PrivacyEcommerce', 'staticpage_Quiosks', 'staticpage_TermsAndConditionsEcommerce', 'staticpage_TermsAndConditionsReturnEcommerce', 'staticpage_black_friday', 'staticpage_club-trocafone', 'staticpage_galaxy-s8', 'staticpage_how-to-buy', 'staticpage

In [6]:
labels['label'].value_counts(normalize=True)

0    0.949521
1    0.050479
Name: label, dtype: float64

In [14]:
rf = RandomForestClassifier(n_estimators=10000, n_jobs=-1, min_samples_split=150,\
                            class_weight='balanced')

#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20)

In [15]:
rf.fit(X_train,Y_train)
rf.score(X_test,Y_test)

0.81380375997939736

In [16]:
Y_pred = rf.predict(X_test)
print(roc_auc_score(Y_test, Y_pred))

0.782421227197


In [17]:
#  Esto nos tira data sobre la precisión, cuántas de las cosas que clasificamos salieron bien clasificadas, y f1

Y_truths = Y_test
print(classification_report(Y_truths, Y_pred))

             precision    recall  f1-score   support

          0       0.98      0.82      0.89      3685
          1       0.18      0.75      0.29       198

avg / total       0.94      0.81      0.86      3883



In [18]:
print(confusion_matrix(Y_truths, Y_pred))

[[3012  673]
 [  50  148]]


In [19]:
feature_importances = pd.DataFrame(rf.feature_importances_,\
                                   index = X_train.columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

In [20]:
feature_importances

,importance
ult_check,2.168837e-01
cant_checkouts,1.330245e-01
prom_timestamp,5.271392e-02
timestamp,5.259321e-02
ult_view,4.656781e-02
min_timestamp,4.314435e-02
check_sku,2.932196e-02
sum_timestamp,2.757911e-02
cant_event,2.485007e-02
event_checkout,2.406693e-02


## Summit

In [21]:
df_submit = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_unidos, how='inner')

In [22]:
df_events

,cant_conversions,cant_checkouts,cant_viewed_product,cant_searched_product,cant_ad_campaign_hit,ad_campaign_hit,cant_lead,lead,cant_brand_listing,cant_visited_site,...,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0.0,1.0,4.0,1.0,0.0,False,0.0,False,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
0297fc1e,0.0,7.0,404.0,6.0,29.0,True,1.0,True,4.0,95.0,...,1,1,1,1,1,1,1,1,1,1
2d681dd8,0.0,1.0,13.0,1.0,1.0,True,0.0,False,5.0,2.0,...,0,0,0,0,1,0,0,0,0,0
cccea85e,0.0,1.0,739.0,1.0,15.0,True,0.0,False,7.0,22.0,...,1,1,0,1,1,1,1,1,1,13
4c8a8b93,0.0,2.0,177.0,9.0,14.0,True,0.0,False,8.0,20.0,...,1,1,0,0,0,0,1,1,1,0
29ebb414,0.0,1.0,8.0,11.0,4.0,True,0.0,False,0.0,4.0,...,1,0,1,1,0,0,0,0,0,0
3dc1950f,0.0,3.0,595.0,2.0,3.0,True,0.0,False,6.0,30.0,...,1,1,1,1,1,1,1,1,1,13
8ea4c165,0.0,3.0,97.0,17.0,13.0,True,0.0,False,5.0,7.0,...,0,0,1,0,1,1,0,0,1,17
d8cfe234,1.0,3.0,39.0,8.0,0.0,False,0.0,False,4.0,4.0,...,1,0,0,0,1,0,0,0,1,15


In [23]:
kaggle_pred = rf.predict_proba(df_events)
kaggle_pred
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [24]:
df_submit.to_csv('submit/submit_clase.csv')


In [25]:
df_submit.label.mean()

0.2576323172596719

In [25]:
0.34460859613130107

0.34460859613130107